In [3]:
from pybtex.database.input import bibtex
import pybtex.errors
import biblib
from urllib.parse import unquote
import os
import bibtexparser
from tqdm import tqdm
import json

pybtex.errors.set_strict_mode(False)

In [5]:
bibtex_file = open("/home/jason/Dokumente/work/webis/ir-anthology/ir-anthology-data/ir-anthology.bib", "r")
bib_database = bibtexparser.load(bibtex_file)

acronym_file=open("/home/jason/Dokumente/work/webis/ir-anthology/ir-anthology-data/criteria.bib","r")
acro_database=bibtexparser.load(acronym_file)

In [18]:
ceph_path="/home/jason/Dokumente/work/webis/ir-anthology/wlgc/papers-by-doi/"
wlgc_new="/home/jason/Dokumente/work/webis/ir-anthology/new_structure/"
new_bibs_json=open("/home/jason/Dokumente/work/webis/ir-anthology/new_structure_jsons/new_bibs_dirs.json","w")
found_json=open("/home/jason/Dokumente/work/webis/ir-anthology/new_structure_jsons/found.json","w")
not_found_json=open("/home/jason/Dokumente/work/webis/ir-anthology/new_structure_jsons/not_found.json","w")
no_doi_json=open("/home/jason/Dokumente/work/webis/ir-anthology/new_structure_jsons/no_doi.json","w")
new_bibs_dirs={}
found_pdf={}
not_found={}
no_doi={}

In [ ]:
for bib_name, bib_entry in tqdm(bib_database.entries_dict.items()):
    if bib_name[0:13] == "CRITERIA:DBLP":
        continue
        
            
    if bib_entry["ENTRYTYPE"]=="proceedings":
        continue
    
    found=0
    try:
        doi = bib_entry["doi"]
    except:
        no_doi[bib_name]=bib_entry
        doi = "0/0"

    doi_url = unquote(doi)
    doi_parts = doi_url.split("/")
    doi_prefix = doi_parts[0]
    doi_suffix = doi_parts[1]

    if doi_prefix=="0":
        continue

    else:
        try:
            for pdf in os.listdir(ceph_path + doi_prefix):
                pdf_url = unquote(pdf)

                if pdf_url[:-4] == doi_suffix:

                    #move to new folder /conference/year/bib_name
                    source_id=bib_entry["sourceid"]
                    conference=source_id.split("/")[1]

                    #print(source_id)
                    #print(source_id.split("/"))
                    try:
                        if conference=="www" or conference=="sigir":
                            if source_id.split("/")[0] == "DBLP:journals":
                                conference_acro=acro_database.entries_dict["CRITERIA:DBLP:"+conference+"journals"]["acronym"]

                            if source_id.split("/")[0] == "DBLP:conf":
                                conference_acro=acro_database.entries_dict["CRITERIA:DBLP:"+conference+"conf"]["acronym"]

                        if conference=="riao":
                            conference_acro="RIAO-OAIR"

                        if conference=="oair":
                            conference_acro="RIAO-OAIR"

                        if conference=="chiir":
                            conference_acro="CHIIR"

                        if conference=="synthesis":
                            conference_acro="Synth-ICR"

                        if conference=="iiix":
                            conference_acro="CHIIR-HCIR"

                        if conference=="hcir":
                            conference_acro="CHIIR-HCIR"

                        if conference=="irs":
                            conference_acro="IRS-series"

                    except:
                        conference_acro=acro_database.entries_dict["CRITERIA:DBLP:"+conference]["acronym"]

                    year=bib_entry["year"]

                    try:
                        os.rename(ceph_path+doi_prefix+"/"+pdf,wlgc_new+str(conference_acro)+"/"+str(year)+"/"+bib_name+".pdf")
                    except:
                        os.makedirs(wlgc_new+str(conference_acro)+"/"+str(year)+"/")
                        os.rename(ceph_path+doi_prefix+"/"+pdf,wlgc_new+str(conference_acro)+"/"+str(year)+"/"+bib_name+".pdf")

                    new_bibs_dirs[bib_name]=wlgc_new+str(conference_acro)+"/"+str(year)+"/"
                    found_pdf[bib_name]=[doi_url,ceph_path + doi_prefix + "/" + pdf]

                    found = 1

            if found == 0:
                # pdf not found
                # looking in subdirs for pdf

                alldirs = [dir[0] for dir in os.walk(ceph_path)]

                for direct in alldirs:
                    for pdf in os.listdir(direct):
                        if pdf.endswith(".pdf"):

                            pdf_url = unquote(pdf)

                            if pdf_url[:-4] == doi_suffix: 
                                # pdf exists

                                os.rename(direct + pdf, bib_name) 

                                try:
                                    os.rename(ceph_path+doi_prefix+"/"+pdf,wlgc_new+str(conference_acro)+"/"+str(year)+"/"+bib_name+".pdf")
                                except:
                                    os.makedirs(wlgc_new+str(conference_acro)+"/"+str(year)+"/")
                                    os.rename(ceph_path+doi_prefix+"/"+pdf,wlgc_new+str(conference_acro)+"/"+str(year)+"/"+bib_name+".pdf")

                                found_pdf[bib_name]=[doi_url,ceph_path + doi_prefix + "/" + pdf]
                                new_bibs_dirs[bib_name]=wlgc_new+str(conference_acro)+"/"+str(year)+"/"

                                found = 1

                        if found == 0:
                            not_found[bib_name]="not found"

        except:
            not_found[bib_name]="not found"



In [20]:
print("found ",len(found_pdf))
print("not found ",len(not_found))
print("no doi ",len(no_doi))
print("new bib dirs ",len(new_bibs_dirs))
json.dump(new_bibs_dirs,new_bibs_json,indent=4, separators=(',', ': '))
json.dump(found_pdf,found_json,indent=4, separators=(',', ': '))
json.dump(not_found,not_found_json,indent=4, separators=(',', ': '))
json.dump(no_doi,no_doi_json,indent=4, separators=(',', ': '))
bibtex_file.close()
new_bibs_json.close()
no_doi_json.close()
not_found_json.close()
found_json.close()

found  27002
not found  22174
no doi  12901
new bib dirs  27002
